<a href="https://colab.research.google.com/github/simon-mellergaard/GAI-with-LLMs/blob/main/Project%20codes/Assignment03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3
> *Evaluate HuggingFaceTB/SmolLM-135M-Instruct on the cnn_dailymail dataset used in Chapter 6 in Natural Language Processing with Transformers.*
> 1. *Compare four different experiments each with a different prompting and decoding strategy. Argue for why these are the most promising ones to try. Evaluate the rouge scores of each model on the test set (if compute becomes a problem use a smaller subset of the test set) and compare to the performance of PEGASUS.*

Todo:


*   Better experiment 3
*   Imrpove code layeout (adding ground truth)
*   Adding only 4500 characters in the pegasus model to make it fair.

## Setup


In [ ]:
# Install packages
!pip install datasets==3.6.0 # To load the cnn dataset
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nv

In [ ]:
# Libraries
import evaluate
import os
import torch
import nltk
import pandas as pd

# Functions
from transformers import pipeline
from google.colab import userdata
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

2025-09-26 12:13:29.287742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758888809.465858      76 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758888809.522015      76 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Setting up the device (GPU).
device = "cuda" if torch.cuda.is_available() else "cpu"

## Loading the dataset



In [ ]:
# Loading the datasets
from datasets import load_dataset

dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0")

README.md: 0.00B [00:00, ?B/s]

cnn_dailymail.py: 0.00B [00:00, ?B/s]

The repository for ccdv/cnn_dailymail contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ccdv/cnn_dailymail.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


cnn_stories.tgz:   0%|          | 0.00/159M [00:00<?, ?B/s]

dailymail_stories.tgz:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# Sampling from the test set
test_sampled = dataset["test"].shuffle(seed=42).select(range(1000)) # modify range. originally 1000

### Test (Should be deleted)

In [ ]:
model_name = "HuggingFaceTB/SmolLM-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
sample_text = dataset["train"][1]["article"][:2000]

In [ ]:
max_length = 2
input_txt = "News:\n\n" + sample_text + "\n\nSummary of the news: "
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
torch.manual_seed(42)
output = model.generate(input_ids, max_new_tokens=10,
                               do_sample=True, top_p=0.8, top_k=50, temperature=0.2, no_repeat_ngram_size=3)
print(tokenizer.decode(output[0]))

News:

(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has now collected eight gold medals at world championships, equaling the record held by American trio Carl Lewis, Michael Johnson and Allyson Felix, not to mention the small matter of six Olympic titles. The relay triumph followed individual successes in the 100 and 200 meters in the Russian capital. "I'm proud of myself and I'll continue to work to dominate for as long as possible," Bolt said, having previously expressed his intention to carry on until the 2016 Rio Olympics. Victory was never

## Experiments with SMolLM

The following [SmolLM-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM-135M-Instruct) model is loaded, which is a decoder-only model:

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In order to create a summary, a promt must be provided to the model, as well as a decoding strategy which relates to how whether or not the output is sampled or not, how many characters to include, hwo to sample etc. This is experimented with in the following ways:

In [ ]:
sample_text = dataset["train"][1]["article"][:2000]
summaries = {}

In [ ]:
# Run this for a reference model
# pipe_gpt = pipeline("text-generation", model="gpt2-xl")

Device set to use cuda:0


In [ ]:
# Another reference model
# pipe = pipeline("text-generation", model="HuggingFaceTB/SmolLM2-135M-Instruct")

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
pipe = pipeline("text-generation", model="HuggingFaceTB/SmolLM-135M-Instruct")

Device set to use cuda:0


In [ ]:
## EXPERIMENT 1 ##
query = "News:\n\n" + sample_text + "\n\nShort summary of the news: "
torch.manual_seed(42)
pipe_out = pipe(query,
                max_new_tokens=128,
                clean_up_tokenization_spaces=True,
                do_sample=True,
                top_p=0.8,
                top_k=50,
                temperature=0.2,
                no_repeat_ngram_size=3)
summaries['EX1'] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(query) :]))
print(summaries['EX1'])

24-year old Usain, who has been a world champion since 21 years, has been declared the world's fastest man by the United Nations.
The United States and the Soviet Union have been competing in the Olympics since 1988, and the two countries have been at the forefront of the world wrestling scene.
The two teams have been in a race for the title for 15 years, and in 22 years, they have been out of the top 12.
The winner of the 30-meter relay has been named as the United Kingdom's Michael Johnson, who was defeated by


In [ ]:
## EXPERIMENT 2 ##
query = sample_text + "\n\nTo sum up: "
torch.manual_seed(42)
pipe_out = pipe(query,
                max_new_tokens=128,
                clean_up_tokenization_spaces=True,
                do_sample=True,
                top_p=0.9,
                top_k=100,
                temperature=0.2,
                no_repeat_ngram_size=3)
summaries['EX2'] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(query) :]))
print(summaries['EX2'])

1.
The United States has won the 300-meter gold medal in the women's 1992 Olympics.
2.
The US has won 12 gold medals in the Olympic Games.
3.
The USA has won three gold medals, including the 500 m gold medal.
4.
The Americans have won 2 golds, including 1 gold medal, in the Olympics.


In [ ]:
## EXPERIMENT 3 ##
query = sample_text + "\nProvide a summary of the news article: "
torch.manual_seed(42)
pipe_out = pipe(query,
                max_new_tokens=128,
                clean_up_tokenization_spaces=True,
                do_sample=True,
                top_p=0.9,
                top_k=30,
                temperature=0.2,
                no_repeat_ngram_size=3)
summaries['EX3'] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(query) :]))
print(summaries['EX3'])

1.
The United States has a new record of three gold medals in the women's 150m and 160m sprint events.
2.
The Olympic Games will be held in Tokyo, Japan, from September 23 to 27.
3.
The US will be the first country to win a gold medal in the Women's 250 meters sprint event.
4.
The Japanese team will be awarded the gold medal for their 24th Olympic title.
5.
The Australian team will also be awarded a gold and silver medal.
6.
The New Zealand team will win


In [ ]:
## EXPERIMENT 4 ##
query = "The text:\n" + sample_text + "\nExtract key points from the text: "
torch.manual_seed(42)
pipe_out = pipe(query,
                max_new_tokens=128,
                clean_up_tokenization_spaces=True,
                do_sample=True,
                top_p=0.9,
                top_k=50,
                temperature=0.1,
                no_repeat_ngram_size=3)
summaries['EX4'] = "\n".join(sent_tokenize(pipe_out[0]['generated_text'][len(query) :]))
print(summaries['EX4'])

1.
The United States was the first team to win the gold medal in the women's 150m sprint.
2.
The Jamaican team was the second team to reach the gold.
3.
The team was able to complete the 300-meter relay in 12.5 seconds.
4.
The Jamaica team was defeated by the United Kingdom in the first round of the 5000 m sprint.
Extract the main points from this passage: 5.
The goal was to reach 110 meters.
6.
The American team was unable to complete their 3


Comparison of all experiments

In [ ]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .

EX1
24-year old Usain, who has been a world champion since 21 years, has been declared the world's fastest man by the United Nations.
The United States and the Soviet Union have been competing in the Olympics since 1988, and the two countries have been at the forefront of the world wrestling scene.
The two teams have been in a race for the title for 15 years, and in 22 years, they have been out of the top 12.
The winner of the 30-meter relay has been named as the United Kingdom's Michael Johnson, who was defeated by

EX2
1.
The United States has won the 300-meter gold medal in the women's 1992 Olympics.
2.
The US has won 12 gold medals in the Olympic Games.
3.
The USA has won three gold medals, including the 500 m gold medal.
4.
The Americans have won 2 golds, including 1 gold medal, in the Oly

In [ ]:
# Loading the rouge metric
rouge_metric = evaluate.load("rouge")

reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
    records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
EX1,0.138462,0.015625,0.107692,0.138462
EX2,0.139535,0.023810,0.093023,0.139535
EX3,0.148760,0.016807,0.099174,0.132231
EX4,0.168067,0.017094,0.100840,0.168067


## Evaluating on test set

In [ ]:
# Arguments for function
model_ckpt = "HuggingFaceTB/SmolLM-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForCausalLM.from_pretrained(model_ckpt).to(device)
batch_size = 8
metric = rouge_metric
generation_args={'length_penalty':0.8, 'num_beams':8, 'max_length':128}

# Evaluation function
article_batches = list(chunks(test_sampled['article'], batch_size))
target_batches = list(chunks(test_sampled['highlights'], batch_size))

prompted_article_batch = ['The text:\n' + article[:4500] + '\nExtract key points from the text: ' for article in article_batches[1]]

inputs = tokenizer(prompted_article_batch, truncation=True,
            padding="max_length", padding_side='left', return_tensors="pt")

summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device),
                **generation_args)

decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True)[len(prompted_article_batch[i]) :]
    for i, s in enumerate(summaries)]

metric.add_batch(predictions=decoded_summaries, references=target_batches[1])

score = metric.compute()

{'length_penalty': 0.8, 'num_beams': 8, 'max_length': 128}

In [ ]:
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries(dataset, metric, model, tokenizer,
                               batch_size=8, device=device,
                               column_text="article",
                               column_summary="highlights",
                               prompt_before="",
                               prompt_after="",
                               generation_args={}):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        # Add prompts to the article batch and truncate
        prompted_article_batch = [prompt_before + article[:4500] + prompt_after for article in article_batch]

        inputs = tokenizer(prompted_article_batch, max_length=1024,  truncation=True,
                        padding="max_length",
                        padding_side='left',
                        return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         **generation_args) # Pass generation_args to model.generate

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True)[len(prompted_article_batch[i]) :]
            for i, s in enumerate(summaries)]

        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

In [ ]:
# Sampling from the test set
test_sampled_small = dataset["test"].shuffle(seed=42).select(range(3000)) # modify range. originally 1000

model_ckpt = "HuggingFaceTB/SmolLM-135M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForCausalLM.from_pretrained(model_ckpt).to(device)

scores = {}

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
# EX 1
generation_args = {'max_new_tokens':128,
                'do_sample':True,
                'top_p':0.8,
                'top_k':50,
                'temperature':0.2,
                'no_repeat_ngram_size':3}

scores['EX1'] = evaluate_summaries(test_sampled_small, rouge_metric, model, tokenizer,
                                   batch_size=8, prompt_before="News:\n\n",
                                   prompt_after="\n\nShort summary of the news: ")

100%|██████████| 375/375 [10:10<00:00,  1.63s/it]


In [ ]:
# EX 2
generation_args = {'max_new_tokens':128,
                'do_sample':True,
                'top_p':0.9,
                'top_k':100,
                'temperature':0.2,
                'no_repeat_ngram_size':3}

scores['EX2'] = evaluate_summaries(test_sampled_small, rouge_metric, model, tokenizer,
                                   batch_size=8, prompt_before="",
                                   prompt_after="\n\nTo sum up: ")

100%|██████████| 375/375 [10:10<00:00,  1.63s/it]


In [ ]:
# EX 3
generation_args = {'max_new_tokens':128,
                'do_sample':True,
                'top_p':0.9,
                'top_k':30,
                'temperature':0.2,
                'no_repeat_ngram_size':3}
prompt_before = ""
prompt_after = "\nProvide a summary of the news article: "

scores['EX3'] = evaluate_summaries(test_sampled_small, rouge_metric, model, tokenizer,
                                   batch_size=8, prompt_before=prompt_before,
                                   prompt_after=prompt_after)

100%|██████████| 375/375 [10:14<00:00,  1.64s/it]


In [ ]:
# EX 4
generation_args = {'max_new_tokens':128,
                'do_sample':True,
                'top_p':0.9,
                'top_k':50,
                'temperature':0.1,
                'no_repeat_ngram_size':3}
prompt_before = "The text:\n"
prompt_after = "\nExtract key points from the text: "

scores['EX4'] = evaluate_summaries(test_sampled_small, rouge_metric, model, tokenizer,
                                   batch_size=8, prompt_before=prompt_before,
                                   prompt_after=prompt_after)

100%|██████████| 375/375 [10:14<00:00,  1.64s/it]


The scores can be compared in a dataframe to see the difference in performance

In [ ]:
scores_df = pd.DataFrame.from_dict(scores, orient='index')
display(scores_df)

,rouge1,rouge2,rougeL,rougeLsum
EX1,0.199817,0.054724,0.142815,0.176748
EX2,0.191505,0.044718,0.139186,0.171731
EX3,0.125595,0.013914,0.098386,0.116794
EX4,0.200439,0.053336,0.145341,0.179600


## Comparing with PEGASUS

To compare with PEGASUS, much of the same code from chapter 6 in NLP is used.

In [ ]:
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               batch_size=8, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score_pegasus = evaluate_summaries_pegasus(test_sampled_small, rouge_metric,
                                   model, tokenizer, batch_size=4)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 750/750 [1:19:46<00:00,  6.38s/it]


In [ ]:
rouge_dict = dict((rn, score_pegasus[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.428212,0.209838,0.305021,0.369485
